In [1]:
from torch.utils.data import DataLoader

from src.concept_bottleneck.dataset import CUB200ImageToClass

batch_size = 16
num_workers = 2

training_data = CUB200ImageToClass(train=True)
test_data = CUB200ImageToClass(train=False)

training_dataloader = DataLoader(training_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)


In [2]:
import torch

from src.concept_bottleneck.networks import get_mlp

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = get_mlp().to(device)


Using device: cuda


In [3]:
import numpy as np


def train(
    model: torch.nn.Module,
    dataloader: DataLoader[tuple[torch.Tensor, np.int_]],
    trained_image_to_attributes_model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: str,
):
    model.train()
    trained_image_to_attributes_model.eval()
    size = len(dataloader.dataset)  # type: ignore
    for batch, (x, y) in enumerate(dataloader):
        x = trained_image_to_attributes_model(x.to(device))
        y = y.to(device)

        logits = model(torch.sigmoid(x))
        loss = loss_fn(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            print(f"loss: {loss.item():>7f} [{batch * len(x):>5d}/{size:>5d}]")


def test(
    model: torch.nn.Module,
    dataloader: DataLoader[tuple[torch.Tensor, np.int_]],
    trained_image_to_attributes_model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    device: str,
):
    model.eval()
    trained_image_to_attributes_model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for x, y in dataloader:
            x = trained_image_to_attributes_model(x.to(device))
            y = y.to(device)

            logits = model(torch.sigmoid(x))
            test_loss += loss_fn(logits, y).item()
            correct += (logits.argmax(dim=1) == y).sum().item()

    test_loss /= len(dataloader)
    accuracy = correct / len(dataloader.dataset)  # type: ignore

    return test_loss, accuracy


In [4]:
from src.concept_bottleneck.train import TrainFn, TestFn, run_epochs, MODEL_PATH
from src.concept_bottleneck.inference import (
    load_image_to_attributes_model,
    INDEPENDENT_IMAGE_TO_ATTRIBUTES_MODEL_NAME,
    SEQUENTIAL_ATTRIBUTES_TO_CLASS_MODEL_NAME,
)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

trained_image_to_attributes_model = load_image_to_attributes_model(
    INDEPENDENT_IMAGE_TO_ATTRIBUTES_MODEL_NAME, device
)

train_fn: TrainFn = lambda model: train(
    model,
    training_dataloader,
    trained_image_to_attributes_model,
    loss_fn,
    optimizer,
    device,
)
test_fn: TestFn = lambda model, dataloader: test(
    model, dataloader, trained_image_to_attributes_model, loss_fn, device
)

epochs = 300


def on_better_accuracy(model: torch.nn.Module, accuracy: float):
    print(
        f"Saving model to {SEQUENTIAL_ATTRIBUTES_TO_CLASS_MODEL_NAME} with accuracy {100 * accuracy:>0.4f}%"
    )
    torch.save(
        model.state_dict(), MODEL_PATH / SEQUENTIAL_ATTRIBUTES_TO_CLASS_MODEL_NAME
    )


run_epochs(
    epochs,
    model,
    train_fn,
    test_fn,
    training_dataloader,
    test_dataloader,
    on_better_accuracy,
)


Using cache found in /home/shuangwu/.cache/torch/hub/pytorch_vision_v0.10.0


Epoch 1/300-------------------
loss: 5.259455 [    0/ 5994]
loss: 4.585021 [ 1600/ 5994]
loss: 3.966138 [ 3200/ 5994]
loss: 3.679837 [ 4800/ 5994]
Training Loss: 3.1757, Training Accuracy: 28.9623%
Test Loss: 3.2713, Test Accuracy: 27.4077%
Saving model to sequential_attributes_to_class.pth with accuracy 27.4077%
Epoch 2/300-------------------
loss: 3.398179 [    0/ 5994]
loss: 3.328391 [ 1600/ 5994]
loss: 2.590011 [ 3200/ 5994]
loss: 2.794323 [ 4800/ 5994]
Training Loss: 2.5594, Training Accuracy: 41.0577%
Test Loss: 2.6922, Test Accuracy: 37.0038%
Saving model to sequential_attributes_to_class.pth with accuracy 37.0038%
Epoch 3/300-------------------
loss: 2.721333 [    0/ 5994]
loss: 2.234967 [ 1600/ 5994]
loss: 2.177254 [ 3200/ 5994]
loss: 2.360098 [ 4800/ 5994]
Training Loss: 2.2694, Training Accuracy: 44.7114%
Test Loss: 2.4241, Test Accuracy: 40.1622%
Saving model to sequential_attributes_to_class.pth with accuracy 40.1622%
Epoch 4/300-------------------
loss: 2.019807 [    0/ 5

OrderedDict([('0.weight',
              tensor([[-0.2298, -2.1506,  0.3203,  ..., -0.8812, -0.6840, -1.3753],
                      [-0.2751, -1.2831, -0.1890,  ...,  0.2126, -0.3411, -2.3089],
                      [-0.4301, -2.2276, -0.2320,  ..., -0.9727, -0.5543, -1.5005],
                      ...,
                      [-0.0326,  2.7551, -0.1022,  ...,  1.9437,  1.5272, -3.4759],
                      [ 0.7784,  0.2427, -0.0144,  ...,  2.4265,  1.3833, -2.0807],
                      [-0.1244,  0.2149, -0.1332,  ..., -0.1947, -2.2526,  1.5636]],
                     device='cuda:0')),
             ('0.bias',
              tensor([ 4.8965e-01,  1.0197e+00, -1.7708e+00,  1.1732e+00,  1.2392e+00,
                       1.4425e+00,  2.8775e-01,  1.1928e+00,  2.6838e+00, -2.5148e-01,
                       5.1124e-01,  3.9156e-01,  3.6574e-01,  6.7374e-01, -7.5921e-01,
                       7.2005e-01,  6.0430e-01,  2.3309e+00, -1.4895e+00, -2.0056e+00,
                      -1.0687e